__AASB_Motifs Probabilísticos__

Motifs probabilísticos baseados num alinhamento múltiplo

In [2]:
import re
from math import log
from threading import local
from typing import List, Type
from unittest import result

In [10]:
#Lista de sequências a testar
lista_seqs = ['ATTG','ATCG','ATTC','ACTC']

seqs2 = """HEM13 CCCATTGTTCTC
HEM13 TTTCTGGTTCTC
HEM13 TCAATTGTTTAG
ANB1 CTCATTGTTGTC
ANB1 TCCATTGTTCTC
ANB1 CCTATTGTTCTC
ANB1 TCCATTGTTCGT
ROX1 CCAATTGTTTTG"""

seqs2 = [x.split()[1] for x in seqs2.splitlines()]

In [33]:
def pwm(lista_seqs: List[str], pseudo: int = 0, printing: bool = True):
    """    
    Input: lista_seqs = lista de sequências
    pseudo = valor da pseudocontagem do PWM (por default é 0)
    printing = modo de retorno da matriz
        
    Output: perfil PWM das sequências
    """
    tam_seqs = len(lista_seqs)
    res = []
    for z in zip(*lista_seqs):
        res.append({k: (z.count(k) + pseudo) / (tam_seqs + (pseudo * 4)) for k in 'ACGT'})
    
    if printing == True:
        bases = sorted(res[0].keys())    
        mat = [[f"{p[b]:-6.3f}" for b in bases] for p in res]    
        for p in zip(*([bases] + mat)):
            print (*p)
    else:
        return res

In [77]:
p = pwm(lista_seqs, pseudo = 0.5, printing = True)

A  0.750  0.083  0.083  0.083
C  0.083  0.250  0.250  0.417
G  0.083  0.083  0.083  0.417
T  0.083  0.583  0.583  0.083


In [76]:
def pssm(lista_seqs: List[str], pseudo: int = 1, printing: bool = True):
    """
    Inputs:
    Input: lista_seqs = lista de sequências
    pseudo = valor da pseudocontagem do PSSM (por default é 1)
    printing = modo de retorno da matriz
          
    Output: perfil PSSM das sequências
    """
    num_seqs = len(lista_seqs)
    res = []
    for z in zip(*lista_seqs):
        res.append({k: log((z.count(k) + pseudo) * 4 / (num_seqs + (pseudo * 4)), 2) for k in "ACGT"})
    
    if printing == True:
        bases = sorted(res[0].keys())    
        mat = [[f"{p[b]:-6.3f}" for b in bases] for p in res]   
        for p in zip(*([bases] + mat)):
            print(*p)
    else:
        return res

In [75]:
pssm(lista_seqs, pseudo = 1, printing = True)

A  1.322 -1.000 -1.000 -1.000
C -1.000  0.000  0.000  0.585
G -1.000 -1.000 -1.000  0.585
T -1.000  1.000  1.000 -1.000


In [79]:
def seq_mais_prov(lista_seqs: List[str], seq: str, pseudo: int = 0):
    """
    Inputs:
    lista_seqs = lista de sequências
    seq = sequência
    pseudo = valor da pseudocontagem para o PWM (por default é 0)    
        
    Output:
    seq_mais_provavel = Sequência mais provável de ocorrer
    """
    def pwm(lista_seqs,pseudo=0):
        tam_seqs = len(lista_seqs)
        res = []
        for z in zip(*lista_seqs):
            res.append({k: (z.count(k) + pseudo) / (tam_seqs + (pseudo * 4)) for k in "ACGT"})
        return res
    
    def prov_seq(seq,pwm):
        assert len(seq)==len(pwm)
        res = 1
        for s in range(len(seq)):
            res *= pwm[s][seq[s]]
        return res
    
    def mais_prov(seq,P):
        chave=[]
        value=[]
        for s in re.findall('(?=(....))', seq):
            chave.append(s)
            value.append(prov_seq(s, pwm = P))
        dic= dict(zip(chave, value))
        return f'{max(dic, key=dic.get)}', f'{dic[max(dic, key=dic.get)]}'
        
    P=pwm(lista_seqs,pseudo)
    return mais_prov(seq,P)

In [80]:
seq_mais_prov(lista_seqs,seq = "TATTTACTCTTTTAGATGTTCTGGACTCAATTGCTCACTCTTCTCTACTCGTTTAGATGATAGTAGCGCTATTGACTCCGATCTTACTCTTTATGATCGATCGATGCACTCATCGTACGTAGATGCTTTT", pseudo = 0)

('ATTG', '0.28125')